# Sweep regions

Extract swept regions around peaks

In [1]:
import re, os, sys, pickle, pickle, subprocess, bisect, random
from pathlib import Path
import numpy
import pandas
from pandas import DataFrame, Series

import scipy

from ete3 import Tree
from scipy.cluster.hierarchy import dendrogram, linkage

import pyfaidx

# my own libaries
from GenomicWindows import window
import GenomicIntervals

numpy.random.seed(7)
random.seed(7)

In [2]:
import re, os, sys, pickle, math, collections, random
from pathlib import Path
import numpy
import pandas
from pandas import DataFrame
from ete3 import Tree
from scipy.cluster.hierarchy import dendrogram, linkage
import subprocess
import bisect
import pyfaidx

Plotting setup:

In [3]:
%matplotlib inline

# Make inline plots vector graphics instead of raster graphics
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina', 'png')

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D

from mpl_toolkits.basemap import Basemap
#matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)

import mpld3

import seaborn as sns
sns.set() # sets seaborn default "prettyness:
sns.set_style("whitegrid")
sns.set_context("paper")

# lowess for plotting
from statsmodels.nonparametric.smoothers_lowess import lowess

set1 = {'red': '#e41a1c', 'blue': '#377eb8', 'green': '#4daf4a',
        'purple': '#984ea3', 'orange': '#ff7f00', 
        'yellow': '#ffff33', 'brown': '#a65628'}

Ignore deprecation warnings from mainly seaborn:

In [4]:
# silence deprecation warnings (lots from seaborn)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=numpy.VisibleDeprecationWarning)
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

### Analysis dirs

In [5]:
root_dir = Path(os.environ['HOME'], 'simons/faststorage/people/kmt')
meta_data_dir = Path(os.environ['HOME'], 'simons/faststorage/data/metadata')
steps_dir = root_dir / 'steps'
argweaver_dir = steps_dir / 'argweaver/output'
results_dir = root_dir / 'results'
figures_dir = root_dir / 'figures'
data_dir = root_dir / 'data'
pi_dir = steps_dir / 'pi_stores'
dist_dir = steps_dir / 'dist_stores'
#pi_dir = root_dir / 'old_pi_stores'
male_x_haploid_dir = steps_dir / 'male_x_haploids'

reference_genome_file = Path('/home', 'kmt', 'simons', 
                        'faststorage', 'cteam_lite_public3', 'FullyPublic', 'Href.fa')

Import variables global to the entire analysis:

In [6]:
import analysis_globals

Local code in the scripts dir on the cluster:

In [7]:
scripts_dir = root_dir / 'scripts'
if str(scripts_dir) not in sys.path:
    sys.path.append(str(scripts_dir))

import simons_meta_data
import hg19_chrom_sizes

from toggle_code_and_errors import toggle_code_html, toggle_errors_html

## Convenience functions

In [8]:
def silent_nanmean(x):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        return numpy.nanmean(x)
    
def ident_scalar(s):
    x = s.unique()
    assert(len(x)) == 1
    return x[0]

def flatten_column_index(df):
    df.columns = ['_'.join(col).strip() for col in df.columns.values]


In [9]:
import time
class Timer:    
    def __enter__(self):
        self.start = time.clock()
        return self

    def __exit__(self, *args):
        self.end = time.clock()
        self.interval = self.end - self.start
        print("{:.0f} min {:.1f} sec".format(self.interval//60, self.interval % 60))

## Load meta data

In [10]:
# easy loading of meta data in a consistent manner across code
individuals, populations, regions = simons_meta_data.get_meta_data(meta_data_dir=meta_data_dir)

pop_categories = pandas.read_hdf(str(results_dir / 'population_categories.store'), 'sr')
region_categories = pandas.read_hdf(str(results_dir / 'region_categories.store'), 'sr')

region_colors = dict(zip(list(region_categories), 
                         ['#e41a1c', '#377eb8',  '#984ea3', '#4daf4a',
                          '#ff7f00', '#ffff33', '#a65628']))

chromosome_lengths = dict((k.replace('chr', ''), v) for k, v in hg19_chrom_sizes.hg19_chrom_sizes.items())

## Load sweep data

In [11]:
non_afr_total_swept = pandas.read_hdf(results_dir / 'non_afr_total_swept.hdf')
afr_total_swept = pandas.read_hdf(results_dir / 'afr_total_swept.hdf')
sweep_peaks = pandas.read_hdf(results_dir / 'sweep_peaks.hdf')
sweep_data = pandas.read_hdf(results_dir / 'sweep_data.hdf')

In [12]:
sweep_peaks.head()

,start,prop_swept,end,pos
run_id,,,,
1,11200000,0.357143,11400000,11300000.0
3,14700000,0.421429,14800000,14750000.0
5,19800000,0.828571,19900000,19850000.0
6,21200000,0.821429,21300000,21250000.0
12,36200000,0.671429,36300000,36250000.0


In [13]:
non_afr_total_swept.head()

,index,start,end,is_non_african,nr_swept,total,pos,prop_swept
0,1,0,100000,True,0.0,140,50000.0,0.0
1,3,100000,200000,True,0.0,140,150000.0,0.0
2,5,200000,300000,True,0.0,140,250000.0,0.0
3,7,300000,400000,True,0.0,140,350000.0,0.0
4,9,400000,500000,True,0.0,140,450000.0,0.0


Make sure the index is 0, 1, 2 ...

In [14]:
non_afr_total_swept.reset_index(drop=True, inplace=True)
print(len(non_afr_total_swept))
non_afr_total_swept.tail()

1552


,index,start,end,is_non_african,nr_swept,total,pos,prop_swept
1547,3095,154700000,154800000,True,0.0,140,154750000.0,0.0
1548,3097,154800000,154900000,True,0.0,140,154850000.0,0.0
1549,3099,154900000,155000000,True,0.0,140,154950000.0,0.0
1550,3101,155000000,155100000,True,0.0,140,155050000.0,0.0
1551,3103,155100000,155200000,True,0.0,140,155150000.0,0.0


For each peak find a region around the peak that spans 1Mb and includes as many individuals as possible.

In [15]:
def get_extended_region(df, peak_pos, max_width, min_sub_prop):
    
    peak_idx = non_afr_total_swept.loc[lambda df: (df.start <= peak_pos) & (df.end > peak_pos)].index[0]
    start_idx, end_idx = peak_idx, peak_idx
    peak_prop_swept = df.prop_swept[start_idx]
    cutoff = peak_prop_swept
    prev_cutoff = peak_prop_swept
    while df.end[end_idx] - df.start[start_idx] < max_width:
    
        # if largest of left and right falls below cutoff we break:
        if max(df.prop_swept[start_idx-1], df.prop_swept[end_idx+1]) <= peak_prop_swept * min_sub_prop:
            # we still want this to represent the peak
            break
        
        # extend to the left:
        if df.prop_swept[start_idx-1] > df.prop_swept[end_idx+1]:
            cutoff = df.prop_swept[start_idx-1]
#             if cutoff > prev_cutoff: # if cutoff incease we are overlapping a different sweep
            if cutoff > prev_cutoff and df.prop_swept[start_idx-2] > prev_cutoff: # if cutoff incease we are overlapping a different sweep
                break
            start_idx -= 1
        # extend to the right:
        else:
            cutoff = df.prop_swept[end_idx+1]
#             if cutoff > prev_cutoff: # if cutoff incease we are overlapping a different sweep
            if cutoff > prev_cutoff and df.prop_swept[end_idx+2] > prev_cutoff:
                break
            end_idx += 1

        prev_cutoff = cutoff
            
    return df.start[start_idx], df.end[end_idx], prev_cutoff

## Get the extended regions around peaks

Extract the region around each peak where some percentage of the peak prop_swept is included.

**Note:** in the dataframes for such regions `prop_swept` does *not* refer to the peak but to a minimum proportion of individuals swept across the region.

In [16]:
for min_sub_prop in [0.1, 0.75, 0.9]:

    extended_regions = list()

    for tup in sweep_peaks.itertuples():

        # get extended region
        start_pos, end_pos, prop_swept = get_extended_region(non_afr_total_swept, tup.pos, 1e9, min_sub_prop)
        
        # get the relevant windows of the chromosome
        df = sweep_data.loc[lambda df: (df.start >= start_pos) & \
                            (df.end <= end_pos) & \
                            (df.swept)]

        # get swept the relevant windows of the chromosome: 
        # get only windows from individuals that are swept across the entire region
        df = df.groupby('indiv_1').filter(lambda df: len(df.index) == (end_pos - start_pos)/100000)

        # get the individuals
        swept_indivs = df.indiv_1.unique().tolist()

        # sanity check
        expected = len(swept_indivs)*(end_pos - start_pos)/100000
        assert expected == len(df.index)
        
        extended_regions.append((tup.pos, tup.start, tup.end, tup.prop_swept, start_pos, end_pos, prop_swept, swept_indivs))

        
    extended_peak_regions = DataFrame().from_records(extended_regions, 
                                   columns=['pos', 'peak_start', 'peak_end', 'peak_prop_swept', 'start_pos', 'end_pos', 
                                            'prop_swept', 'swept_indivs'])

    extended_peak_regions.swept_indivs = [repr(x) for x in extended_peak_regions.swept_indivs]
    extended_peak_regions.to_hdf(results_dir / 'extended_peak_regions_{:.0f}%.hdf'.format(min_sub_prop*100), 'df', format='table', mode='w')



Write one of the sets of extended regions to pickle file for using for dating them:

In [17]:
extended_peak_regions_10 = pandas.read_hdf(results_dir / 'extended_peak_regions_10%.hdf')
(extended_peak_regions_10
 .drop('swept_indivs', axis=1)
 .sort_values(by='pos')
 .assign(prop_swept=lambda df: df.prop_swept.round(2),
        pos=lambda df: df.pos.astype(int))
 .to_csv(results_dir / 'extended_peak_regions_10%.csv', index=False)
)
extended_peak_regions_75 = pandas.read_hdf(results_dir / 'extended_peak_regions_75%.hdf')
(extended_peak_regions_75
 .drop('swept_indivs', axis=1)
 .sort_values(by='pos')
 .assign(prop_swept=lambda df: df.prop_swept.round(2),
        pos=lambda df: df.pos.astype(int))
 .to_csv(results_dir / 'extended_peak_regions_75%.csv', index=False)
)
extended_peak_regions_90 = pandas.read_hdf(results_dir / 'extended_peak_regions_90%.hdf')
(extended_peak_regions_90
 .drop('swept_indivs', axis=1)
 .sort_values(by='pos')
 .assign(prop_swept=lambda df: df.prop_swept.round(2),
        pos=lambda df: df.pos.astype(int))
 .to_csv(results_dir / 'extended_peak_regions_90%.csv', index=False)
)

In [18]:
"Total 90% region", extended_peak_regions_90.assign(length = lambda df: df.end_pos-df.start_pos).length.sum()

('Total 90% region', 13000000)

In [19]:
"Total 75% region", extended_peak_regions_75.assign(length = lambda df: df.end_pos-df.start_pos).length.sum()

('Total 75% region', 19000000)

In [20]:
10700000 / 155270560

0.06891196888837137

In [21]:
with sns.color_palette([set1['blue']]):
    with sns.axes_style("whitegrid", {'axes.grid' : False}):
        fig, ax = plt.subplots(figsize=(10, 4))

        x = [j for i in zip(non_afr_total_swept.start, non_afr_total_swept.end) for j in i]
        y = [j for i in zip(non_afr_total_swept.prop_swept, non_afr_total_swept.prop_swept) for j in i]      
        g = ax.plot(x, y)
        g = ax.scatter(sweep_peaks.pos, sweep_peaks.prop_swept, c='red')

        for tup in extended_peak_regions_90.itertuples():
            g = ax.add_patch(Rectangle((tup.start_pos, 0), tup.end_pos-tup.start_pos, tup.prop_swept, 
                         facecolor='red',
                         linewidth=0,
                         alpha=1.0,
                         zorder=1))

        for tup in extended_peak_regions_75.itertuples():
            g = ax.add_patch(Rectangle((tup.start_pos, -0.02), tup.end_pos-tup.start_pos, 0.02, 
                         facecolor='black',
                         linewidth=0,
                         alpha=1.0,
                         zorder=1))
            
        ax.set_ylim(-0.02,1)
        ax.set_xlim(0,160e6)

        ax.set_ylabel('Proportion of non-Africans called as ECH')
        ax.set_xlabel('Chromosomal position')
        
        plt.tight_layout()
        
        plt.savefig(str(figures_dir / 'tot_swept_ext_regions.pdf')) 
        
#mpld3.display(fig)

In [22]:
(extended_peak_regions_90
 .assign(prop_swept_across=lambda df: [len(eval(x))/140 for x in df.swept_indivs])
 .assign(length=lambda df: df.end_pos-df.start_pos)
 .drop('swept_indivs', axis=1)
 .sort_values('length', ascending=False)
)

,pos,peak_start,peak_end,peak_prop_swept,start_pos,end_pos,prop_swept,prop_swept_across,length
22,110900000.0,110700000,111100000,0.992857,110200000,111100000,0.950000,0.914286,900000
17,77150000.0,77100000,77200000,0.564286,76700000,77400000,0.521429,0.507143,700000
2,19850000.0,19800000,19900000,0.828571,19600000,20200000,0.757143,0.671429,600000
24,127050000.0,127000000,127100000,0.728571,126800000,127400000,0.678571,0.600000,600000
28,154200000.0,154000000,154400000,0.321429,153900000,154400000,0.300000,0.300000,500000
3,21250000.0,21200000,21300000,0.821429,21100000,21600000,0.778571,0.735714,500000
5,37500000.0,37300000,37700000,0.521429,37200000,37700000,0.514286,0.514286,500000
6,43300000.0,43100000,43500000,0.292857,43100000,43600000,0.278571,0.278571,500000
7,49900000.0,49800000,50000000,0.564286,49500000,50000000,0.521429,0.521429,500000
8,51050000.0,50900000,51200000,0.435714,50800000,51300000,0.414286,0.407143,500000


In [23]:
(extended_peak_regions_75
 .assign(prop_swept_across=lambda df: [len(eval(x))/140 for x in df.swept_indivs])
 .assign(length=lambda df: df.end_pos-df.start_pos)
 .drop('swept_indivs', axis=1)
 .sort_values('length', ascending=False)
)

,pos,peak_start,peak_end,peak_prop_swept,start_pos,end_pos,prop_swept,prop_swept_across,length
22,110900000.0,110700000,111100000,0.992857,109400000,111200000,0.757143,0.528571,1800000
3,21250000.0,21200000,21300000,0.821429,20700000,21700000,0.657143,0.528571,1000000
2,19850000.0,19800000,19900000,0.828571,19500000,20300000,0.642857,0.535714,800000
4,36250000.0,36200000,36300000,0.671429,35900000,36700000,0.521429,0.314286,800000
12,64850000.0,64800000,64900000,0.821429,64400000,65200000,0.635714,0.435714,800000
27,132750000.0,132600000,132900000,0.485714,132400000,133100000,0.371429,0.257143,700000
24,127050000.0,127000000,127100000,0.728571,126800000,127500000,0.607143,0.521429,700000
23,114150000.0,114100000,114200000,0.585714,113800000,114500000,0.457143,0.371429,700000
10,55500000.0,55400000,55600000,0.414286,55200000,55900000,0.342857,0.200000,700000
17,77150000.0,77100000,77200000,0.564286,76700000,77400000,0.521429,0.507143,700000


In [24]:
(extended_peak_regions_10
 .assign(prop_swept_across=lambda df: [len(eval(x))/140 for x in df.swept_indivs])
 .assign(length=lambda df: df.end_pos-df.start_pos)
 .drop('swept_indivs', axis=1)
 .sort_values('length', ascending=False)
)

,pos,peak_start,peak_end,peak_prop_swept,start_pos,end_pos,prop_swept,prop_swept_across,length
22,110900000.0,110700000,111100000,0.992857,109100000,111600000,0.214286,0.114286,2500000
24,127050000.0,127000000,127100000,0.728571,126000000,127700000,0.114286,0.050000,1700000
12,64850000.0,64800000,64900000,0.821429,63800000,65300000,0.185714,0.035714,1500000
9,54350000.0,54300000,54400000,0.785714,53400000,54900000,0.092857,0.000000,1500000
4,36250000.0,36200000,36300000,0.671429,35700000,37100000,0.085714,0.007143,1400000
11,62950000.0,62900000,63000000,0.607143,62500000,63900000,0.185714,0.007143,1400000
15,73200000.0,73100000,73300000,0.650000,72300000,73600000,0.085714,0.000000,1300000
2,19850000.0,19800000,19900000,0.828571,19300000,20600000,0.314286,0.021429,1300000
27,132750000.0,132600000,132900000,0.485714,132300000,133600000,0.214286,0.021429,1300000
10,55500000.0,55400000,55600000,0.414286,55000000,56300000,0.085714,0.000000,1300000


In [25]:
extended_peak_regions_10 = pandas.read_hdf(results_dir / 'extended_peak_regions_10%.hdf')
(extended_peak_regions_10
 .drop('swept_indivs', axis=1)
 .sort_values(by='pos')
 .assign(prop_swept=lambda df: df.prop_swept.round(2),
        pos=lambda df: df.pos.astype(int))
 .to_csv(results_dir / 'extended_peak_regions_10%.csv', index=False)
)

In [33]:
renaming = {'peak_start': 'peak start', 'peak_prop_swept': 'peak prop swept', 'peak_end': 'peak end',
            'start_pos': 'start', 'end_pos': 'end', 
            'prop_swept': 'Min freq. ECH', 'prop_swept_across': 'Freq. ECHs spanning region'}

df = ((extended_peak_regions_75 
       .assign(prop_swept=lambda df: df.prop_swept.round(2),
               prop_swept_across=lambda df: [round(len(eval(x))/140, 2) for x in df.swept_indivs],
               length=lambda df: df.end_pos - df.start_pos)
       .drop(['pos', 'swept_indivs'], axis=1)
      ).rename(columns=renaming)
      .merge((extended_peak_regions_90
             .assign(prop_swept=lambda df: df.prop_swept.round(2),
                     prop_swept_across=lambda df: [round(len(eval(x))/140, 2) for x in df.swept_indivs],
                     length=lambda df: df.end_pos - df.start_pos)
              .drop(['pos', 'swept_indivs'], axis=1)
             ).rename(columns=renaming), 
             on=['peak start', 'peak end', 'peak prop swept'],
             suffixes=(' (75%)', ' (90%)'))
     .sort_values(by='peak start')
     )

#df.to_csv(results_dir / 'ech_regions_som.csv', index=False)
df

,peak start,peak end,peak prop swept,start (75%),end (75%),Min freq. ECH (75%),length (75%),Freq. ECHs spanning region (75%),start (90%),end (90%),Min freq. ECH (90%),length (90%),Freq. ECHs spanning region (90%)
0,11200000,11400000,0.357143,11100000,11700000,0.31,600000,0.28,11100000,11500000,0.33,400000,0.32
1,14700000,14800000,0.421429,14400000,15000000,0.32,600000,0.22,14700000,14900000,0.41,200000,0.41
2,19800000,19900000,0.828571,19500000,20300000,0.64,800000,0.54,19600000,20200000,0.76,600000,0.67
3,21200000,21300000,0.821429,20700000,21700000,0.66,1000000,0.53,21100000,21600000,0.78,500000,0.74
4,36200000,36300000,0.671429,35900000,36700000,0.52,800000,0.31,36000000,36400000,0.64,400000,0.63
5,37300000,37700000,0.521429,37200000,37700000,0.51,500000,0.51,37200000,37700000,0.51,500000,0.51
6,43100000,43500000,0.292857,43100000,43600000,0.28,500000,0.28,43100000,43600000,0.28,500000,0.28
7,49800000,50000000,0.564286,49500000,50000000,0.52,500000,0.52,49500000,50000000,0.52,500000,0.52
8,50900000,51200000,0.435714,50800000,51400000,0.34,600000,0.33,50800000,51300000,0.41,500000,0.41
9,54300000,54400000,0.785714,53900000,54400000,0.66,500000,0.66,54000000,54400000,0.75,400000,0.75


## Table for supplementary

In [35]:
df = (((sweep_peaks[['pos', 'start', 'end', 'prop_swept']]
        .merge(extended_peak_regions_90, suffixes=('_peak', ''), 
               left_on=['pos', 'start', 'end'], right_on=['pos', 'peak_start', 'peak_end'])
       ).drop(['start', 'end'], axis=1)
        .assign(prop_swept_peak=lambda df: [round(x, 2) for x in df.prop_swept_peak] )

       .assign(prop_swept=lambda df: df.prop_swept.round(2),
               prop_swept_across=lambda df: [round(len(eval(x))/140, 2) for x in df.swept_indivs],
               peak_start = lambda df: (df.peak_start / 100000).astype('int'),
               peak_end = lambda df: (df.peak_end / 100000).astype('int'),
               start_pos=lambda df: (df.start_pos / 100000).astype('int'),
               end_pos=lambda df: (df.end_pos / 100000).astype('int'))
       .assign(length=lambda df: df.end_pos - df.start_pos)
       .drop(['swept_indivs'], axis=1)
      )[['pos', 'peak_start', 'peak_end', 'prop_swept_peak', 'start_pos', 'end_pos', 'length', 'prop_swept', 'prop_swept_across']]
      .merge((extended_peak_regions_75
             .assign(prop_swept=lambda df: df.prop_swept.round(2),
                     prop_swept_across=lambda df: [round(len(eval(x))/140, 2) for x in df.swept_indivs],
                     peak_start = lambda df: (df.peak_start / 100000).astype('int'),
                     peak_end = lambda df: (df.peak_end / 100000).astype('int'),
                     start_pos=lambda df: (df.start_pos / 100000).astype('int'),
                     end_pos=lambda df: (df.end_pos / 100000).astype('int'))
              .assign(length=lambda df: df.end_pos - df.start_pos)
              .drop(['swept_indivs'], axis=1)
             )[['pos', 'peak_start', 'peak_end', 'start_pos', 'end_pos', 'length', 'prop_swept', 'prop_swept_across']], 
             on=['pos', 'peak_start', 'peak_end'],
             suffixes=(' (90%)', ' (75%)'))
     .sort_values(by=['peak_start'])
     )

df.columns = pandas.MultiIndex(levels=[['Peak', '90%-region', '75%-region'], 
                                       ['pos', 'Start', 'End', 'Freq. ECH', 'Length', 'Min freq. ECH', 'Freq. ECHs spanning region']], 
                               labels=[[0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2], 
                                       [0, 1, 2, 3, 1, 2, 4, 5, 6, 1, 2, 4, 5, 6]])
df = df.drop(('Peak', 'pos'), axis=1)

df[('Data S1', 'Subplot')] = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ$%&#@*'[:len(df)])

df.to_csv(results_dir / 'ech_regions_som_multidx.csv', index=False)
df

Peak                 90%-region                             \
   Start   End Freq. ECH      Start   End Length Min freq. ECH   
0    112   114      0.36        111   115      4          0.33   
1    147   148      0.42        147   149      2          0.41   
2    198   199      0.83        196   202      6          0.76   
3    212   213      0.82        211   216      5          0.78   
4    362   363      0.67        360   364      4          0.64   
5    373   377      0.52        372   377      5          0.51   
6    431   435      0.29        431   436      5          0.28   
7    498   500      0.56        495   500      5          0.52   
8    509   512      0.44        508   513      5          0.41   
9    543   544      0.79        540   544      4          0.75   
10   554   556      0.41        553   558      5          0.39   
11   629   630      0.61        627   631      4          0.55   
12   648   649      0.82        647   651      4          0.77   
13   668   670      0.47        667   670      3          0.46   
14   674   676      0.27        674   678      4          0.26   
15   731   733      0.65        730   735      5          0.59   
16   743   744      0.52        740   744      4          0.48   
17   771   772      0.56        767   774      7          0.52   
18   987   988      0.75        985   989      4          0.72   
19  1009  1014      0.29       1009  1014      5          0.29   
20  1048  1050      0.28       1047  1050      3          0.27   
21  1067  1069      0.34       1067  1069      2          0.34   
22  1107  1111      0.99       1102  1111      9          0.95   
23  1141  1142      0.59       1140  1143      3          0.55   
24  1270  1271      0.73       1268  1274      6          0.68   
25  1297  1301      0.65       1297  1302      5          0.64   
26  1313  1314      0.59       1312  1316      4          0.58   
27  1326  1329      0.49       1326  1329      3          0.49   
28  1540  1544      0.32       1539  1544      5          0.30   

                              75%-region                             \
   Freq. ECHs spanning region      Start   End Length Min freq. ECH   
0                        0.32        111   117      6          0.31   
1                        0.41        144   150      6          0.32   
2                        0.67        195   203      8          0.64   
3                        0.74        207   217     10          0.66   
4                        0.63        359   367      8          0.52   
5                        0.51        372   377      5          0.51   
6                        0.28        431   436      5          0.28   
7                        0.52        495   500      5          0.52   
8                        0.41        508   514      6          0.34   
9                        0.75        539   544      5          0.66   
10                       0.30        552   559      7          0.34   
11                       0.51        626   632      6          0.52   
12                       0.75        644   652      8          0.64   
13                       0.45        666   671      5          0.39   
14                       0.26        674   679      5          0.24   
15                       0.58        730   735      5          0.59   
16                       0.46        739   745      6          0.42   
17                       0.51        767   774      7          0.52   
18                       0.69        985   991      6          0.59   
19                       0.29       1009  1014      5          0.29   
20                       0.27       1045  1051      6          0.21   
21                       0.32       1067  1072      5          0.29   
22                       0.91       1094  1112     18          0.76   
23                       0.52       1138  1145      7          0.46   
24                       0.60       1268  1275      7          0.61   
25                       0.64       1297

## relative TMRCA_half in extended regions

We use the 90% set for this and only regions with enough individuals swept that we call them as sweeps.


In [36]:
tmrca_stats_world = pandas.read_hdf(results_dir / 'pruned_tmrca_stats.hdf')
tmrca_stats_world.head()

,start,end,chain,MCMC_sample,coal_half,tmrca,tmrca_half,rel_tmrca_half
0,0,100000,1.5,2750.0,226.957517,45083.337393,12645.212660,0.281658
1,100000,200000,1.5,2750.0,362.848331,47935.198573,14351.381341,0.300756
2,200000,300000,1.5,2750.0,551.114128,53009.238578,19172.584913,0.363367
3,300000,400000,1.5,2750.0,495.469935,50038.851797,14837.471573,0.297887
4,400000,500000,1.5,2750.0,513.143674,50557.752598,18956.576133,0.376540


In [37]:
extended_peak_regions_90.loc[lambda df: df.peak_prop_swept >= analysis_globals.min_prop_swept]

,pos,peak_start,peak_end,peak_prop_swept,start_pos,end_pos,prop_swept,swept_indivs
2,19850000.0,19800000,19900000,0.828571,19600000,20200000,0.757143,"['B_Australian-4', 'B_Crete-2', 'B_Dai-4', 'B_..."
3,21250000.0,21200000,21300000,0.821429,21100000,21600000,0.778571,"['B_Australian-4', 'B_Crete-2', 'B_French-3', ..."
4,36250000.0,36200000,36300000,0.671429,36000000,36400000,0.642857,"['B_Crete-2', 'B_Dai-4', 'B_French-3', 'B_Han-..."
5,37500000.0,37300000,37700000,0.521429,37200000,37700000,0.514286,"['B_Crete-2', 'B_Dai-4', 'B_Han-3', 'B_Karitia..."
7,49900000.0,49800000,50000000,0.564286,49500000,50000000,0.521429,"['B_Australian-4', 'B_French-3', 'B_Papuan-15'..."
9,54350000.0,54300000,54400000,0.785714,54000000,54400000,0.750000,"['B_Australian-4', 'B_Crete-2', 'B_Dai-4', 'B_..."
11,62950000.0,62900000,63000000,0.607143,62700000,63100000,0.550000,"['B_Crete-2', 'B_Dai-4', 'S_Adygei-1', 'S_Aleu..."
12,64850000.0,64800000,64900000,0.821429,64700000,65100000,0.771429,"['B_Crete-2', 'B_Dai-4', 'B_French-3', 'B_Han-..."
15,73200000.0,73100000,73300000,0.650000,73000000,73500000,0.585714,"['B_French-3', 'B_Han-3', 'B_Papuan-15', 'B_Sa..."
16,74350000.0,74300000,74400000,0.521429,74000000,74400000,0.478571,"['B_Crete-2', 'B_French-3', 'B_Sardinian-3', '..."


In [38]:
global_mean = tmrca_stats_world.rel_tmrca_half.mean()
print('global mean rel_tmrca_half:', global_mean)

global mean rel_tmrca_half: 0.151110848696


In [39]:
def get_rel_tmrca_half(df):
    s, e = df.start_pos.iloc[0], df.end_pos.iloc[0]
    return tmrca_stats_world.loc[lambda df: (df.start >= s) & (df.end <= e)]

In [40]:
sr = (extended_peak_regions_90.loc[lambda df: df.prop_swept >= 0.25]
           .groupby(['pos', 'prop_swept']).apply(get_rel_tmrca_half).reset_index()
          )['rel_tmrca_half']
print('mean rel_tmrca_half for all sweeps:', sr.mean())
print(scipy.stats.ttest_ind(tmrca_stats_world.rel_tmrca_half, sr))

mean rel_tmrca_half for all sweeps: 0.0596192864686
Ttest_indResult(statistic=9.3515037159822025, pvalue=2.6623199946911374e-20)


In [41]:
sr = (extended_peak_regions_90.loc[lambda df: df.peak_prop_swept >= analysis_globals.min_prop_swept]
           .groupby(['pos', 'prop_swept']).apply(get_rel_tmrca_half).reset_index()
          )['rel_tmrca_half']
print('mean rel_tmrca_half for all sweeps:', sr.mean())
print(scipy.stats.ttest_ind(tmrca_stats_world.rel_tmrca_half, sr))

mean rel_tmrca_half for all sweeps: 0.0431899341069
Ttest_indResult(statistic=8.9470530575185521, pvalue=9.6629134985021742e-19)


## Extract SNPs shared by all swept individuals (for David)

We use the 90% set for this and only regions with enough individuals swept that we call them as sweeps.

We extract window for all non-Africans and get all positions segregating in non-Africans that are fixed in the swept non-African individuals. We discard positions with more than one base, with less than half of all bases called or with less than half of bases called in swept individuals.

Load sequence indexes:

In [42]:
file_index = dict()
for file_path in male_x_haploid_dir.glob('*.fa'):
    name = file_path.name.split('.')[0]
    file_index[name] = pyfaidx.Fasta(str(file_path), 
                                     strict_bounds=True, 
                                     one_based_attributes=False)
    
reference_index = pyfaidx.Fasta(str(reference_genome_file), 
                                 strict_bounds=True, 
                                 one_based_attributes=False)
    
def get_alignmnet(sample_ids, start, end):
    assert start < end
    start, end = int(start), int(end)
    seq_list = list()
    name_list = list()

    for sample_id in sample_ids:
        name = sample_id + '-A'   
        seq_list.append(file_index[name]['X'][start:end].seq)
        name_list.append(name.replace('-A', ''))
        
    return name_list, seq_list

In [ ]:
from collections import Counter

non_afr_indivs = sweep_data.loc[lambda df: df.region_1 != 'Africa', 'indiv_1'].unique()

records = list()
for tup in extended_peak_regions_90.loc[lambda df: df.peak_prop_swept >= analysis_globals.min_prop_swept].itertuples():

    pos, start, end = tup.pos, tup.peak_start, tup.peak_end

    # sequenes in window
    names, aln = get_alignmnet(non_afr_indivs, start, end)    

    # reference sequence
    ref = reference_index['X'][start:end].seq
    
    # get snp columns
    snp_idx = dict()
    for j in range(end-start):
        col = [aln[i][j] for i in range(len(aln))]

        if col.count('N') / len(non_afr_indivs) > 0.5:
            continue

        counter = Counter(col)

        if 'N' in counter:
            del counter['N']

        counts = counter.most_common()
        if len(counts) == 2:
            snp_idx[j] = counter

    # get snps that are fixed in swept indivs
    swept_indivs = eval(tup.swept_indivs)
    subaln = [seq for (name, seq) in zip(names, aln) if name in swept_indivs]
    for j, counter in snp_idx.items():
        try:
            col = [subaln[i][j] for i in range(len(subaln))]
        except IndexError:
            print(j, len(subaln), len(swept_indivs))
            print(names)
            print(swept_indivs)
            print
            raise

        if col.count('N') / len(swept_indivs) > 0.5:
            continue

        bases = set(col)
        if 'N' in bases:
            bases.remove('N')
        if len(bases) == 1:
            swept_base = bases.pop()
            alt_base = [b for b in counter if b != swept_base][0]
            ref_base = ref[j]            
            records.append((pos, start, end,
                  j+start,
                  swept_base, counter[swept_base], 
                  alt_base, counter[alt_base], 
                  ref_base, 
                  len(swept_indivs), len(non_afr_indivs), repr(swept_indivs)))
        
snps_for_david = pandas.DataFrame().from_records(records, columns=['peak_pos', 'start', 'end', 'snp_pos', 
                                                  'swept_base', 'swept_base_count', 'other_base', 'other_base_count', 
                                                  'ref_base', 'nr_swept_indiv', 'nr_non_afr_indiv', 'swept_indivs'])

In [ ]:
snps_for_david.to_hdf(results_dir / 'snps_for_david.hdf', 'df', format='table', mode='w')
snps_for_david.to_csv(results_dir / 'snps_for_david.tsv', sep='\t')